Import of libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # To standardize the data
import cvxpy as cp

Import of the HMLasso function

In [2]:
### Adapt the path "C:/Users/Kilian/Desktop/ENSAE/STATAPP" to run the cell

import sys
sys.path.insert(1, 'C:/Users/Kilian/Desktop/ENSAE/STATAPP/Projet_Statapp/pretreatment')

import file_04_HMLasso as hml

## Data downloading and separation of the dataset

Dataset containing the types of each column from data_03.csv

In [3]:
columns_types = pd.read_csv("data_03_columns_types.csv", index_col=0)
columns_types.head(3)

,Name,Type
0,HHIDPN,Cont
1,HHID,Char
2,PN,Char


Downloading the data with social and genetic variables.

In [4]:
data = pd.read_csv("data_03.csv")

C:\Users\Kilian\AppData\Local\Temp\ipykernel_6808\3952530018.py:1: DtypeWarning: Columns (2684) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data_03.csv")


The column "genetic_Section_A_or_E" have mixed types, so we change its format.

In [5]:
temporary = np.where(data['genetic_Section_A_or_E'] == 'E', 1, np.where(data['genetic_Section_A_or_E'] == 'A', 0, np.nan))

In [6]:
data["genetic_Section_A_or_E"] = temporary

Now we add the health index created by t-SNE

In [7]:
tSNE_GHI = pd.read_csv("data_tSNE_GHI.csv")

We merge the t-SNE health index to the data

In [8]:
data = data.merge(tSNE_GHI, how ='left', on ='HHIDPN')

The final outcome to predict is tSNE_GHI14, so we only keep individuals who were interviewed during the last wave (14th wave)

In [9]:
data_bis = data[data['tSNE_GHI14'].notna()]

Number of individuals present in every waves.

In [10]:
tSNE_GHI[~tSNE_GHI.isnull().any(axis=1)].shape[0]

3396

We select the outcome tSNE_GHI

In [11]:
Y = data_bis[["tSNE_GHI" + str(i) for i in range (1,15)]]

We drop the previous health index GHIw from the data, which won't be used as outcome.
(list_columns_GHI contains the names of GHIw columns).

We drop the outcome to create the matrix X.

In [12]:
X = data_bis.drop(["GHI" + str(i) for i in range (1,15)], axis = 1)
X.drop(["tSNE_GHI" + str(i) for i in range (1,15)], axis = 1, inplace =True)

Now we split the dataset into training, validation and test sets.

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=18)
X_test, X_valid, Y_test, Y_valid = train_test_split(X_test, Y_test, test_size=0.5, random_state = 6)

Smaller sets while coding

In [14]:
nb_test, nb_train, nb_valid = len(X_test.index)//10, len(X_train.index)//10, len(X_valid.index)//10
X_test, Y_test = X_test.iloc[:nb_test], Y_test.iloc[:nb_test]
X_train, Y_train = X_train.iloc[:nb_train], Y_train.iloc[:nb_train]
X_valid, Y_valid = X_valid.iloc[:nb_valid], Y_valid.iloc[:nb_valid]

In [15]:
X_test.to_csv("X_test.csv", index=False)
Y_test.to_csv("Y_test.csv", index=False)
X_train.to_csv("X_train.csv", index=False)
Y_train.to_csv("Y_train.csv", index=False)
X_valid.to_csv("X_valid.csv", index=False)
Y_valid.to_csv("Y_valid.csv", index=False)

In [167]:
#If needed
X_test = pd.read_csv("X_test.csv", index_col=0)
Y_test = pd.read_csv("Y_test.csv", index_col=0)
X_train = pd.read_csv("X_train.csv", index_col=0)
Y_train = pd.read_csv("Y_train.csv", index_col=0)
X_valid = pd.read_csv("X_valid.csv", index_col=0)
Y_valid = pd.read_csv("Y_valid.csv", index_col=0)

## Machine learning

In [25]:
# The objective here is to make a dataset where we observe if each variable exists at each wave
temporal_variables = {}
waves_columns = [col for col in X_train.columns if "genetic_" not in col and col[1] in "123456789"]
for col in waves_columns:
  char = col[0] # R or H
  if col[2] in "01234":
    wave = col[1:3]
    suffix = col[3:]
  else:
    wave = col[1]
    suffix = col[2:]
  variable = char + 'w' + suffix
  
  if variable not in temporal_variables.keys():
    temporal_variables[variable] = np.zeros((14), dtype=bool)
  
  temporal_variables[variable][int(wave)-1] = True

temporal_variables = pd.DataFrame(temporal_variables)

# We manually add "GHIw":
temporal_variables["tSNE_GHIw"] = np.ones((14), dtype=bool)
waves_columns += [f"t_SNE_GHI{w}" for w in range(1,15)]

In [26]:
# Timeless data
non_waves_columns = [col for col in X_train.columns if col not in waves_columns]

In [27]:
# We put the explaining variables by wave in a list of dataset
# Intemporal variables are put in each one of them
liste = []    # len = 14 
for i in range(14):
    columns_wave_i = [col.replace('w', str(i+1)) for col in temporal_variables.T[i].index[temporal_variables.T[i]] if col != "tSNE_GHIw"]
    liste.append(X_train.loc[X_train["INW"+str(i+1)] == 1, columns_wave_i + non_waves_columns])

In [28]:
X_train1 = liste[0].values #Rajouter var X train en dataframe
Y_train1 = Y_train.iloc[:,0]
Y_train1.dropna(inplace =True)
Y_train1 = Y_train1.values
Y_train1 = (Y_train1 - np.mean(Y_train1))/np.std(Y_train1)
#Y_train1 = Y_train1
#Vérifier meme longueur

In [29]:
scaler = StandardScaler()#(with_std=False)
X_train1 = scaler.fit_transform(X_train1)

C:\Users\Kilian\anaconda3\lib\site-packages\sklearn\utils\extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\Kilian\anaconda3\lib\site-packages\sklearn\utils\extmath.py:990: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
C:\Users\Kilian\anaconda3\lib\site-packages\sklearn\utils\extmath.py:1020: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count


We apply the lasso

In [30]:
hml.ERRORS_HANDLING = "ignore"
lasso = hml.HMLasso(mu = 100)
lasso.fit(X_train1, Y_train1)

[Imputing parameters] Starting...
[Imputing parameters] R calculated.
[Imputing parameters] rho_pair calculated.
[Imputing parameters] S_pair calculated.
[Imputing parameters] Parameters imputed.
[First Problem] Starting...
[First Problem] Objective and constraints well-defined.
                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Apr 22 08:41:47 PM: Your problem has 929296 variables, 1 constraints, and 0 parameters.
(CVXPY) Apr 22 08:41:47 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 22 08:41:47 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 22 08:41:47 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compil

Seuil à 10e-14

In [53]:
coeff = lasso.beta_opt.copy()

In [80]:
coeff = np.abs(coeff)
coeff

array([2.52855022e-11, 1.38769180e-12, 2.32397560e-11, 1.95931350e-13,
       3.41285511e-11, 1.59447484e-11, 4.68248118e-12, 3.64647758e-11,
       1.52100104e-11, 2.32098795e-11, 2.19761735e-11, 1.02961532e-11,
       1.19571041e-11, 1.11891525e-11, 2.99360187e-20, 1.15831705e-11,
       1.01215561e-11, 2.90000523e-11, 2.90292116e-11, 2.90150433e-11,
       2.88613595e-11, 2.89190709e-11, 2.89190676e-11, 7.25043143e-13,
       9.14267115e-12, 9.14267119e-12, 7.84097441e-12, 1.02962399e-11,
       1.42615419e-11, 3.80986736e-12, 1.25994177e-11, 1.75995254e-11,
       4.15135394e-13, 1.57932072e-12, 5.33661538e-12, 1.04997607e-11,
       6.04190647e-13, 7.02717907e-12, 8.93692992e-12, 5.49080284e-12,
       1.04228703e-11, 4.00722289e-13, 9.83361003e-12, 5.91300572e-12,
       4.52895331e-15, 7.04127218e-12, 2.15491149e-12, 4.47684400e-12,
       1.90861701e-12, 4.03732681e-12, 8.53803213e-13, 3.84943668e-12,
       1.34205562e-11, 3.66107723e-11, 1.71726927e-11, 1.67110121e-11,
      

In [77]:
np.quantile(coeff,0.19)

4.93048415048859e-13

In [83]:
bool_vec = coeff < 10**(-14)
bool_vec

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [84]:
columns = liste[0]
selected = columns[columns.columns[~bool_vec]]
selected

,R1MPART,R1MLEN,R1MCURLN,R1MLENM,R1MNEV,H1ANYFIN,H1ANYFAM,R1FAMR,R1FINR,H1HHRESP,...,RABPLACF_1,RADNEPI_0,RADNEPI_1,RADNEPI_2,RADNEPI_3,RADNEPI_5,RADNEPI_8,RAEVBRNF_0.0,RAEVBRNF_1.0,RAEVBRNF_3.0
6954,0.0,26.5,NaN,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10086,1.0,0.0,NaN,0.0,1.0,1.0,1.0,1.0,0.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1063,0.0,31.9,31.9,0.0,0.0,1.0,1.0,1.0,0.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5407,0.0,32.0,32.0,0.0,0.0,1.0,1.0,0.0,1.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8716,0.0,41.3,41.3,0.0,0.0,1.0,1.0,1.0,0.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8552,0.0,29.7,29.7,0.0,0.0,1.0,1.0,1.0,1.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2802,0.0,14.4,NaN,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7356,0.0,27.0,3.4,0.0,0.0,1.0,1.0,1.0,0.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4649,0.0,17.0,12.9,0.0,0.0,1.0,1.0,0.0,0.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [86]:
liste[0]["REXITWV_14.0"]

6954    NaN
10086   NaN
1063    NaN
5407    NaN
8716    NaN
         ..
8552    NaN
2802    NaN
7356    NaN
4649    NaN
3200    NaN
Name: REXITWV_14.0, Length: 315, dtype: float64